In [11]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import sqlite3

In [10]:
with sqlite3.connect(r"../db/database.db") as conn:    
    query = """
        SELECT CRITICO, COUNT(CRITICO) AS TOTAL 
        FROM MATERIAIS 
        GROUP BY CRITICO
        """
    df_critico = pd.read_sql(query, conn)
    
df_critico["TOTAL_STR"] = df_critico["TOTAL"].apply(lambda x : "{:,.0f}".format(x).replace(",", "X").replace(".", ",").replace("X", "."))
df_critico

,CRITICO,TOTAL,TOTAL_STR
0,Não,522756,522.756
1,Sim,125957,125.957


In [7]:
def convert_mes(value):
    meses = {'Jan': '01','Fev': '02','Mar': '03','Abr': '04','Mai': '05','Jun': '06','Jul': '07','Ago': '08','Set': '09','Out': '10','Nov': '11','Dez': '12'}
    month, year = value.split("-")
    return f"{year}-{meses[month.capitalize()]}"

columns_fx = ['Material', 'Texto Breve', 'Centro', 'Descr. Centro','Identificação', 'Mat + Centro', 'Dep. Supr. Ext.', 'Dono do Estoque', 'Dif. Mês Ant.', 'Crítico?', 'Vr. Crít./ Estrat.', 'Cadastro Novo? (2 anos)', 'Vr. Cadastro Novo', 'Obsoleto p/ Reanálse?', 'Vr. Obsoleto', 'Compart ?', 'Compart Manut']

columns_dt = ['Set/23', 'Out/23', 'Nov/23', 'Dez/23', 'Jan/24', 'Fev/24', 'Mar/24', 'Abr/24', 'Mai/24', 'Jun/24', 'Jul/24', 'Ago/24', 'set/24']

df_base = df.melt(id_vars=columns_fx, value_vars=columns_dt, var_name="Periodo", value_name="Valor")
vlr_est = df_base.groupby(by=["Periodo"], as_index=False)["Valor"].sum()
vlr_est["VrEst"] = vlr_est["Valor"].apply(lambda x : "{:,.2f}".format(x).replace(",", "X").replace(".", ",").replace("X", "."))
vlr_est["Periodo"] = vlr_est["Periodo"].apply(lambda x : f"{x.split("/")[0].capitalize()}-{x.split("/")[1]}")

vlr_est["MesStr"] = vlr_est["Periodo"].apply(convert_mes)
vlr_est.sort_values(by="MesStr", inplace=True)
vlr_est


,Periodo,Valor,VrEst,MesStr
11,Set-23,1.340831e+08,"134.083.111,32",23-09
10,Out-23,1.367558e+08,"136.755.793,95",23-10
9,Nov-23,1.393026e+08,"139.302.625,58",23-11
2,Dez-23,1.421248e+08,"142.124.814,68",23-12
4,Jan-24,1.464216e+08,"146.421.583,03",24-01
3,Fev-24,1.474979e+08,"147.497.875,29",24-02
8,Mar-24,1.461288e+08,"146.128.784,12",24-03
0,Abr-24,1.482487e+08,"148.248.705,53",24-04
7,Mai-24,1.484577e+08,"148.457.736,69",24-05
6,Jun-24,1.498448e+08,"149.844.751,29",24-06


In [20]:

columns = {'Material': "MATERIAL", 'Texto Breve': "TXT_CURTO", 'Centro': "CEN", 'Descr. Centro': "DESC_CEN", 'Identificação': "AREA",
       'Dep. Supr. Ext.': "DEP_EXT", 'Dono do Estoque': "DONO_EST", 'Crítico?': "CRITICO", 'Periodo': "PERIODO", 'Valor': "VALOR"}
df_base = df_base.rename(columns=columns)


In [13]:
df_base = pd.read_excel("../Book1.xlsx")
df_base

,TP_OCOR,UNIDADE,DT_OCOR,MES_OCOR,ANO_OCOR,TURNO_OCOR,LOCAL,SETOR,GENERO,FAIXA_IDADE,DESC_OCOR
0,Acidente,Araucária,2023-05-09,Mai,2023,Manhã,Estacionamento,ADM,Feminino,29 - 33,Tropeçou vindo a cair lesionando o tornozelo
1,Incidente,Lages,2023-05-10,Fev,2023,Tarde,Manutenção,Qualidade,Masculino,43 - 48,Escorregou no piso molhado
2,Acidente com Afastamento >15,Lages,2023-07-10,Jun,2023,Tarde,Logística,Produção,Feminino,34 - 38,Lesão no punho
3,Acidente com Afastamento <15,Curitibanos,2024-02-10,Fev,2024,Manhã,Áreas Diversas,Almoxarifado,Masculino,>53,Cortou o dedo
4,Acidente com Afastamento <15,Curitibanos,2024-06-10,Jun,2024,Tarde,Produção,Manutenção,Feminino,39 - 43,Trauma na cabeça
5,Acidente com Afastamento <15,Brasnorte,2024-08-10,Ago,2024,Noite,Estacionamento,SST,Masculino,29 - 33,Lesão corporal na parte superior
6,Acidente com Afastamento <15,Brasnorte,2024-09-10,Set,2024,Noite,Áreas Diversas,RH,Masculino,0 - 18,Machucou o pé
7,Acidente com Afastamento >15,Araucária,2024-09-10,Set,2024,Noite,Produção,Qualidade,Masculino,19 - 23,Machucou o pé


In [14]:
with sqlite3.connect("../db/db_sst.db") as conn:
    df_base.to_sql("database", conn, if_exists="replace")